# ML to predict Bitcoin prices

+ Data from: https://coinmarketcap.com/api/
+ Adapted from: https://dashee87.github.io/data%20science/deep%20learning/python/another-keras-tutorial-for-neural-network-beginners/
+ Adapted from: https://dashee87.github.io/deep%20learning/python/predicting-cryptocurrency-prices-with-deep-learning/
+ Adapted from: https://github.com/dashee87/blogScripts/blob/master/Jupyter/2017-11-20-predicting-cryptocurrency-prices-with-deep-learning.ipynb

In [1]:
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

# get market info for bitcoin from the start of 2016 to the current day
bitcoin_market_info = pd.read_html("https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end="+time.strftime("%Y%m%d"))[0]

# convert the date string to the correct date format
bitcoin_market_info = bitcoin_market_info.assign(Date=pd.to_datetime(bitcoin_market_info['Date']))
# when Volume is equal to '-' convert it to 0
bitcoin_market_info.loc[bitcoin_market_info['Volume']=="-",'Volume']=0
# convert to int
bitcoin_market_info['Volume'] = bitcoin_market_info['Volume'].astype('int64')
# look at the first few rows
bitcoin_market_info.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,2018-04-15,7999.33,8338.42,7999.33,8329.11,5244480000,135812000000
1,2018-04-14,7874.67,8140.71,7846.00,7986.24,5191430000,133682000000
2,2018-04-13,7901.09,8183.96,7758.93,7895.96,7764460000,134114000000
3,2018-04-12,6955.38,7899.23,6806.51,7889.25,8906250000,118048000000
4,2018-04-11,6843.47,6968.32,6817.59,6968.32,4641890000,116126000000


In [2]:
bitcoin_market_info.columns =[bitcoin_market_info.columns[0]]+['bt_'+i for i in bitcoin_market_info.columns[1:]]

In [3]:
# market_info = pd.merge(bitcoin_market_info,eth_market_info, on=['Date'])
# market_info = market_info[market_info['Date']>='2016-01-01']
market_info = bitcoin_market_info
market_info = market_info[market_info['Date']>='2016-01-01']
for coins in ['bt_']: 
    kwargs = { coins+'day_diff': lambda x: (x[coins+'Close']-x[coins+'Open'])/x[coins+'Open']}
    market_info = market_info.assign(**kwargs)
market_info.head()

,Date,bt_Open,bt_High,bt_Low,bt_Close,bt_Volume,bt_Market Cap,bt_day_diff
0,2018-04-15,7999.33,8338.42,7999.33,8329.11,5244480000,135812000000,0.041226
1,2018-04-14,7874.67,8140.71,7846.00,7986.24,5191430000,133682000000,0.014168
2,2018-04-13,7901.09,8183.96,7758.93,7895.96,7764460000,134114000000,-0.000649
3,2018-04-12,6955.38,7899.23,6806.51,7889.25,8906250000,118048000000,0.134266
4,2018-04-11,6843.47,6968.32,6817.59,6968.32,4641890000,116126000000,0.018244


In [4]:
for coins in ['bt_']: 
    kwargs = { coins+'close_off_high': lambda x: 2*(x[coins+'High']- x[coins+'Close'])/(x[coins+'High']-x[coins+'Low'])-1,
            coins+'volatility': lambda x: (x[coins+'High']- x[coins+'Low'])/(x[coins+'Open'])}
    market_info = market_info.assign(**kwargs)

In [5]:
model_data = market_info[['Date']+[coin+metric for coin in ['bt_'] 
                                   for metric in ['Close','Volume','close_off_high','volatility']]]
# need to reverse the data frame so that subsequent rows represent later timepoints
model_data = model_data.sort_values(by='Date')
model_data.head()

,Date,bt_Close,bt_Volume,bt_close_off_high,bt_volatility
835,2016-01-01,434.33,36278900,-0.560641,0.020292
834,2016-01-02,433.44,30096600,0.250597,0.009641
833,2016-01-03,430.01,39633800,-0.173865,0.020827
832,2016-01-04,433.09,38477500,-0.474265,0.012649
831,2016-01-05,431.96,34522600,-0.013333,0.010391


In [6]:
split_date = '2017-06-01'
# we don't need the date columns anymore
training_set, test_set = model_data[model_data['Date']<split_date], model_data[model_data['Date']>=split_date]
training_set = training_set.drop('Date', 1)
test_set = test_set.drop('Date', 1)
test_set

,bt_Close,bt_Volume,bt_close_off_high,bt_volatility
318,2407.88,1653180000,-0.493815,0.069946
317,2488.55,1317030000,-1.000000,0.047932
316,2515.35,1514950000,-0.159278,0.063496
315,2511.81,1355120000,0.111061,0.052339
314,2686.81,1369310000,-1.000000,0.070287
313,2863.20,2089610000,-0.115346,0.114860
312,2732.16,1517710000,0.625637,0.058835
311,2805.62,1281170000,-0.865882,0.053060
310,2823.81,1348950000,0.468564,0.037789
309,2947.71,2018890000,-0.967912,0.072288


In [7]:
window_len = 1
norm_cols = [coin+metric for coin in ['bt_'] for metric in ['Close','Volume']]

In [8]:
print(type(test_set))
print(test_set)
LSTM_training_inputs = []
for i in range(len(training_set)-window_len):
    temp_set = training_set[i:(i+window_len)].copy()
    for col in norm_cols:
        temp_set.loc[:, col] = temp_set[col]/temp_set[col].iloc[0] - 1
    LSTM_training_inputs.append(temp_set)
LSTM_training_outputs = (training_set['bt_Close'][window_len:].values/training_set['bt_Close'][:-window_len].values)-1
print(LSTM_training_inputs)

<class 'pandas.core.frame.DataFrame'>
     bt_Close   bt_Volume  bt_close_off_high  bt_volatility
318   2407.88  1653180000          -0.493815       0.069946
317   2488.55  1317030000          -1.000000       0.047932
316   2515.35  1514950000          -0.159278       0.063496
315   2511.81  1355120000           0.111061       0.052339
314   2686.81  1369310000          -1.000000       0.070287
313   2863.20  2089610000          -0.115346       0.114860
312   2732.16  1517710000           0.625637       0.058835
311   2805.62  1281170000          -0.865882       0.053060
310   2823.81  1348950000           0.468564       0.037789
309   2947.71  2018890000          -0.967912       0.072288
308   2958.11  1752400000          -0.506760       0.053042
307   2659.63  2569530000           0.410612       0.162094
306   2717.02  1781200000           0.038800       0.051721
305   2506.37  1696560000           0.500227       0.137617
304   2464.58  2026260000          -0.564071       0.128722
30

In [9]:
LSTM_training_inputs[0]

,bt_Close,bt_Volume,bt_close_off_high,bt_volatility
835,0.0,0.0,-0.560641,0.020292


In [10]:
LSTM_test_inputs = []
for i in range(len(test_set)-window_len):
    temp_set = test_set[i:(i+window_len)].copy()
    for col in norm_cols:
        temp_set.loc[:, col] = temp_set[col]/temp_set[col].iloc[0] - 1
    LSTM_test_inputs.append(temp_set)
LSTM_test_outputs = (test_set['bt_Close'][window_len:].values/test_set['bt_Close'][:-window_len].values)-1

In [30]:
LSTM_training_inputs = [np.array(LSTM_training_input) for LSTM_training_input in LSTM_training_inputs]
LSTM_training_inputs = np.array(LSTM_training_inputs)

LSTM_test_inputs = [np.array(LSTM_test_inputs) for LSTM_test_inputs in LSTM_test_inputs]
LSTM_test_inputs = np.array(LSTM_test_inputs)
LSTM_test_inputs

[[[ 0.          0.         -0.49381482  0.06994621]]

 [[ 0.          0.         -1.          0.04793201]]

 [[ 0.          0.         -0.1592775   0.0634955 ]]

 ..., 
 [[ 0.          0.         -0.98173366  0.15710428]]

 [[ 0.          0.          0.35519846  0.05379384]]

 [[ 0.          0.          0.04828475  0.03742506]]]


array([[[ 0.        ,  0.        , -0.49381482,  0.06994621]],

       [[ 0.        ,  0.        , -1.        ,  0.04793201]],

       [[ 0.        ,  0.        , -0.1592775 ,  0.0634955 ]],

       ..., 
       [[ 0.        ,  0.        , -0.98173366,  0.15710428]],

       [[ 0.        ,  0.        ,  0.35519846,  0.05379384]],

       [[ 0.        ,  0.        ,  0.04828475,  0.03742506]]])

In [12]:
# import the relevant Keras modules
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout

def build_model(inputs, output_size, neurons, activ_func="linear",
                dropout=0.25, loss="mae", optimizer="adam"):
    model = Sequential()

    model.add(LSTM(neurons, input_shape=(inputs.shape[1], inputs.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model

Using TensorFlow backend.


In [13]:
# random seed for reproducibility
np.random.seed(202)
# we'll try to predict the closing price for the next 5 days 
# change this value if you want to make longer/shorter prediction
pred_range = 1
# initialise model architecture
bt_model = build_model(LSTM_training_inputs, output_size=pred_range, neurons = 20)
# model output is next 5 prices normalised to 10th previous closing price
LSTM_training_outputs = []
for i in range(window_len, len(training_set['bt_Close'])-pred_range):
    LSTM_training_outputs.append((training_set['bt_Close'][i:i+pred_range].values/
                                  training_set['bt_Close'].values[i-window_len])-1)
LSTM_training_outputs = np.array(LSTM_training_outputs)
# train model on data
# note:bt_history contains information on the training error per epoch
bt_history = bt_model.fit(LSTM_training_inputs[:-pred_range], LSTM_training_outputs, 
                            epochs=50, batch_size=1, verbose=2, shuffle=True)

Epoch 1/50
 - 2s - loss: 0.0237
Epoch 2/50
 - 1s - loss: 0.0221
Epoch 3/50
 - 1s - loss: 0.0212
Epoch 4/50
 - 1s - loss: 0.0206
Epoch 5/50
 - 1s - loss: 0.0196
Epoch 6/50
 - 1s - loss: 0.0194
Epoch 7/50
 - 1s - loss: 0.0188
Epoch 8/50
 - 1s - loss: 0.0191
Epoch 9/50
 - 1s - loss: 0.0190
Epoch 10/50
 - 1s - loss: 0.0191
Epoch 11/50
 - 1s - loss: 0.0189
Epoch 12/50
 - 1s - loss: 0.0188
Epoch 13/50
 - 1s - loss: 0.0188
Epoch 14/50
 - 1s - loss: 0.0187
Epoch 15/50
 - 1s - loss: 0.0187
Epoch 16/50
 - 1s - loss: 0.0185
Epoch 17/50
 - 1s - loss: 0.0187
Epoch 18/50
 - 1s - loss: 0.0187
Epoch 19/50
 - 1s - loss: 0.0186
Epoch 20/50
 - 1s - loss: 0.0185
Epoch 21/50
 - 1s - loss: 0.0186
Epoch 22/50
 - 1s - loss: 0.0186
Epoch 23/50
 - 1s - loss: 0.0187
Epoch 24/50
 - 1s - loss: 0.0187
Epoch 25/50
 - 1s - loss: 0.0187
Epoch 26/50
 - 1s - loss: 0.0186
Epoch 27/50
 - 1s - loss: 0.0186
Epoch 28/50
 - 1s - loss: 0.0185
Epoch 29/50
 - 1s - loss: 0.0187
Epoch 30/50
 - 1s - loss: 0.0185
Epoch 31/50
 - 1s -

In [14]:
bt_pred_prices = ((bt_model.predict(LSTM_test_inputs)[:-pred_range][::pred_range]+1)*\
                   test_set['bt_Close'].values[:-(window_len + pred_range)][::pred_range].reshape(int(np.ceil((len(LSTM_test_inputs)-pred_range)/float(pred_range))),1))

In [15]:
bt_pred_prices

array([[  2420.55977847],
       [  2498.0706541 ],
       [  2529.95794281],
       [  2526.7345436 ],
       [  2698.48370845],
       [  2887.58868198],
       [  2754.43389313],
       [  2817.10421454],
       [  2840.73884027],
       [  2960.79979335],
       [  2971.74777472],
       [  2698.63349414],
       [  2732.54294499],
       [  2540.33948867],
       [  2482.34907425],
       [  2530.06983765],
       [  2669.49752208],
       [  2567.62649819],
       [  2605.02101841],
       [  2735.66921441],
       [  2706.02811295],
       [  2715.49811293],
       [  2754.85908673],
       [  2631.98344351],
       [  2606.86286481],
       [  2499.34807098],
       [  2564.63076687],
       [  2586.05066693],
       [  2551.96875963],
       [  2498.25962882],
       [  2450.89060183],
       [  2516.45123054],
       [  2575.38163792],
       [  2612.26321364],
       [  2611.99272912],
       [  2616.75733741],
       [  2566.14266765],
       [  2580.19558279],
       [  25

In [16]:
test = np.array([[[0.,0.,-0.49381482,0.06994621]]])

In [17]:
bt_pred_price = bt_model.predict(test)

In [18]:
bt_pred_price

array([[ 0.00526597]], dtype=float32)

In [19]:
LSTM_test_inputs[0][0]

array([ 0.        ,  0.        , -0.49381482,  0.06994621])

In [20]:
test_price = (bt_model.predict(LSTM_test_inputs))

Querying like above, without the full calculation, returns wrong results. Calculation:

```
bt_pred_prices = ((bt_model.predict(LSTM_test_inputs)[:-pred_range][::pred_range]+1)*\
                   test_set['bt_Close'].values[:-(window_len + pred_range)[::pred_range].reshape(int(np.ceil((len(LSTM_test_inputs)-pred_range)/float(pred_range))),1))
```
Think the solution into read in and prepare data (test_set, LSTM_test_inputs) from scratch using only one days worth of data.

In [21]:
test_price

array([[ 0.00526597],
       [ 0.00382574],
       [ 0.00580747],
       [ 0.0059418 ],
       [ 0.00434483],
       [ 0.00851797],
       [ 0.00815247],
       [ 0.00409324],
       [ 0.005995  ],
       [ 0.00444063],
       [ 0.00461035],
       [ 0.01466503],
       [ 0.00571327],
       [ 0.01355323],
       [ 0.00720979],
       [ 0.00456995],
       [ 0.00512732],
       [ 0.00758806],
       [ 0.00595494],
       [ 0.00509935],
       [ 0.0062951 ],
       [ 0.00372891],
       [ 0.00362456],
       [ 0.00891757],
       [ 0.00674011],
       [ 0.0084319 ],
       [ 0.00477221],
       [ 0.00437348],
       [ 0.00498115],
       [ 0.0070217 ],
       [ 0.00671195],
       [ 0.00398222],
       [ 0.00441557],
       [ 0.00408328],
       [ 0.00384432],
       [ 0.00314248],
       [ 0.01885236],
       [ 0.00344395],
       [ 0.00854062],
       [ 0.00976544],
       [ 0.00626317],
       [ 0.00470291],
       [ 0.00616661],
       [ 0.00869412],
       [ 0.01416343],
       [ 0

In [38]:
# get market info for bitcoin from the start of 2016 to the current day
bitcoin_market_new = pd.read_html("https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20180415&end=20180416")[0]
print(bitcoin_market_new)
# convert the date string to the correct date format
bitcoin_market_new = bitcoin_market_new.assign(Date=pd.to_datetime(bitcoin_market_info['Date']))
print(bitcoin_market_new)
# convert to int
bitcoin_market_new['Volume'] = bitcoin_market_new['Volume'].astype('int64')
# look at the first few rows
bitcoin_market_new.head()

bitcoin_market_new.columns =[bitcoin_market_new.columns[0]]+['bt_'+i for i in bitcoin_market_new.columns[1:]]

market_new = bitcoin_market_new
market_new = market_new[market_new['Date']>='2016-01-01']
for coins in ['bt_']: 
    kwargs = { coins+'day_diff': lambda x: (x[coins+'Close']-x[coins+'Open'])/x[coins+'Open']}
    market_new = market_new.assign(**kwargs)
market_new.head()

for coins in ['bt_']: 
    kwargs = { coins+'close_off_high': lambda x: 2*(x[coins+'High']- x[coins+'Close'])/(x[coins+'High']-x[coins+'Low'])-1,
            coins+'volatility': lambda x: (x[coins+'High']- x[coins+'Low'])/(x[coins+'Open'])}
    market_new = market_new.assign(**kwargs)
    
m_data = market_new[['Date']+[coin+metric for coin in ['bt_'] 
                                   for metric in ['Close','Volume','close_off_high','volatility']]]
# need to reverse the data frame so that subsequent rows represent later timepoints
m_data = m_data.sort_values(by='Date')
m_data.head()

predict_set = m_data.drop('Date', 1)

print(type(predict_set))
print(predict_set)

window_len = 1
norm_cols = [coin+metric for coin in ['bt_'] for metric in ['Close','Volume']]

LSTM_predict_inputs = []

for i in range(len(predict_set)):
    temp_set = predict_set[i:(i+window_len)].copy()
    for col in norm_cols:
        temp_set.loc[:, col] = temp_set[col]/temp_set[col].iloc[0] - 1
    LSTM_predict_inputs.append(temp_set)
LSTM_test_outputs = (test_set['bt_Close'][window_len:].values/test_set['bt_Close'][:-window_len].values)-1

LSTM_predict_inputs = [np.array(LSTM_predict_inputs) for LSTM_predict_inputs in LSTM_predict_inputs]
LSTM_predict_inputs = np.array(LSTM_predict_inputs)

print(LSTM_predict_inputs)
print(type(LSTM_predict_inputs))

           Date     Open     High      Low    Close      Volume    Market Cap
0  Apr 15, 2018  7999.33  8338.42  7999.33  8329.11  5244480000  135812000000
        Date     Open     High      Low    Close      Volume    Market Cap
0 2018-04-15  7999.33  8338.42  7999.33  8329.11  5244480000  135812000000
<class 'pandas.core.frame.DataFrame'>
   bt_Close   bt_Volume  bt_close_off_high  bt_volatility
0   8329.11  5244480000          -0.945088        0.04239
[[[ 0.          0.         -0.94508832  0.0423898 ]]]
<class 'numpy.ndarray'>


In [23]:
bt_test_price = ((bt_model.predict(LSTM_predict_inputs)[:-pred_range][::pred_range]+1)*\
                   test_set['bt_Close'].values[:-(window_len + pred_range)][::pred_range].reshape(int(np.ceil((len(LSTM_predict_inputs)-pred_range)/float(pred_range))),1))

ValueError: cannot reshape array of size 317 into shape (834,1)

In [ ]:
bt_test_price

In [34]:
bt_model.predict(LSTM_predict_inputs)

array([[ 0.00374119]], dtype=float32)

In [33]:
(bt_model.predict(LSTM_predict_inputs)[:-pred_range][::pred_range]+1)*\
                   test_set['bt_Close'].values[:-(window_len + pred_range)][::pred_range]

array([], shape=(0, 317), dtype=float64)